# Cold Face Test – Saliva Plots

In [ ]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps
import biopsykit as bp
from biopsykit.utils.dataframe_handling import multi_xs
from biopsykit.protocols import MIST

from cft_analysis.datasets import CftDatasetProcessed

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

## Data Import

In [ ]:
# get path to analysis results
base_path = Path("../../data")

In [ ]:
results_path = base_path.joinpath("../results")
stats_path = results_path.joinpath("statistics")
tex_path = stats_path.joinpath("tex_tables")
plot_path = results_path.joinpath("plots")
bp.utils.file_handling.mkdirs([results_path, stats_path, plot_path, tex_path])

paper_path = Path("../paper_path.json")
paper_tex_path = None
paper_img_path = None
if paper_path.exists():
    paper_path = Path(json.load(paper_path.open(encoding="utf-8"))["paper_path"])
    paper_tex_path = paper_path.joinpath("tab")
    paper_img_path = paper_path.joinpath("img")
    bp.utils.file_handling.mkdirs([paper_tex_path, paper_img_path])

In [ ]:
dataset = CftDatasetProcessed(base_path, exclude_subjects=True)
dataset

In [ ]:
hue_order = ["Control", "CFT"]

In [ ]:
mist = MIST.from_file(base_path.joinpath("mist_cft.json"))
mist

In [ ]:
mist.add_saliva_data(dataset.cortisol, "cortisol", sample_times=dataset.sample_times)

## Plots

### Cortisol Response

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))

mist.saliva_plot(
    "cortisol",
    legend_loc="upper right",
    legend_fontsize="medium",
    hue_order=hue_order,
    linestyle=["-", "--"],
    marker=["o", "P"],
    ax=ax,
)

for path in [plot_path, paper_img_path]:
    if path is not None:
        fig.savefig(path.joinpath("img_cortisol_response.pdf"), transparent=True)

### Cortisol Features

#### Prepare Data

In [ ]:
features = ["auc_g", "auc_i", "auc_i_post", "max_inc", "slopeS1S4"]

cort_analysis = multi_xs(dataset.cortisol_features, features, level="saliva_feature")
cort_analysis.head()

#### Statistics

In [ ]:
steps = [
    ("prep", "normality"),
    ("test", "pairwise_ttests"),
]
params = {"dv": "cortisol", "between": "condition", "groupby": "saliva_feature", "test__parametric": False}

stats = bp.stats.StatsPipeline(steps, params)
stats.apply(cort_analysis)
stats.display_results(prep=False)

#### Boxplots

In [ ]:
features = {"auc_g": ["auc_g"], "auc_i": ["auc_i"], "max_inc": ["max_inc"], "slope": ["slopeS1S4"]}

box_pairs, pvalues = stats.sig_brackets(
    "test", stats_effect_type="between", plot_type="multi", x="saliva_feature", features=features, subplots=True
)

fig, axs = plt.subplots(ncols=len(features.keys()))

bp.protocols.plotting.saliva_multi_feature_boxplot(
    dataset.cortisol_features,
    "cortisol",
    features=features,
    hue="condition",
    hue_order=hue_order,
    legend_loc="upper center",
    legend_orientation="horizontal",
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues},
    palette=cmaps.faculties,
    axs=axs,
)

for path in [plot_path, paper_img_path]:
    if path is not None:
        fig.savefig(path.joinpath("img_cortisol_features.pdf"), transparent=True)